In [1]:
import pandas as pd
import numpy as np
import spacy
import nltk
import seaborn
import matplotlib.pyplot as plt
import unidecode
from sklearn.metrics import jaccard_score, log_loss
import re
import pattern

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/Leeman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 100

In [4]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train = train.dropna()
test = test.fillna(" ")

In [5]:
contractions_dict = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [6]:
from autocorrect import Speller

spell = Speller(lang='en')
print(spell('shiwer'))

shiver


In [13]:
re.sub("\[math\].*\[\/math\]" , ' ',train['question1'])

TypeError: expected string or bytes-like object

In [59]:
## This block takes a few minutes

import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)


stop_words = set(nltk.corpus.stopwords.words('english'))
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

def clean(text):
    #remove html/things in boxes
    text = re.sub("\[math\].*\[\/math\]" , ' ', text)
    text = unidecode.unidecode(text)
    text = str.lower(text)
    text = re.sub("-", " ", text)
    text = re.sub("\(.*\)", " ", text)
    text = expand_contractions(text)
    text = re.sub("[\?\#\'\",.!:]+", "", text)
    for s in sym_spell.lookup_compound(text, max_edit_distance=2):
        text = s.term
    tokens = nltk.tokenize.word_tokenize(text)
    
    tokens = [lemmatizer.lemmatize(w) for w in tokens if not w in stop_words] 
  
    return tokens


    

#train['question1_cleaned'] = train['question1'].apply(clean)
#train['question2_cleaned'] = train['question2'].apply(clean)
#test['question1_cleaned'] = test['question1'].apply(clean)
#test['question2_cleaned'] = test['question2'].apply(clean)

In [58]:
for s in sym_spell.lookup_compound("paper mker", max_edit_distance=2):
    print(type(s.term))

<class 'str'>


In [33]:
dir(symspellpy)

['SymSpell',
 'Verbosity',
 '__author__',
 '__author_email__',
 '__builtins__',
 '__cached__',
 '__description__',
 '__doc__',
 '__file__',
 '__license__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__title__',
 '__version__',
 'editdistance',
 'helpers',
 'symspellpy']

In [37]:
import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# lookup suggestions for single-word input strings
input_term = "memebers"  # misspelling of "members"
# max edit distance per lookup
# (max_edit_distance_lookup <= max_dictionary_edit_distance)
suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST,
                               max_edit_distance=2)
# display suggestion term, term frequency, and edit distance
for suggestion in suggestions:
    print(suggestion)

members, 1, 226656153


In [62]:
import time
start = time.time()
print(train.head(100)['question1'].map(clean))
print(time.time()-start)

0                                                                              [step, step, guide, invest, share, market, india]
1                                                                                                     [story, kohinoor, diamond]
2                                                                            [increase, speed, internet, connection, using, van]
3                                                                                                      [mentally, lonely, solve]
4                                                         [one, dissolve, water, quickly, sugar, salt, methane, carbon, dioxide]
5                                                                       [astrology, capricorn, sun, cap, moon, cap, rising, say]
6                                                                                                                    [buy, iago]
7                                                                                                

In [64]:
(len(train) + len(test))/100/60/60

7.639119444444445

In [ ]:
train['question2_cleaned'] = train['question2'].apply(clean)
test['question1_cleaned'] = test['question1'].apply(clean)
test['question2_cleaned'] = test['question2'].apply(clean)

In [ ]:
train.to_csv('train_cleaned.csv')
test.to_csv('test_cleaned.csv')

In [ ]:
train.head()

In [ ]:
def jaccard(row):
    s1 = set(row['question1_cleaned'])
    s2 = set(row['question2_cleaned'])
    u = set(s1 | s2)
    i = set(s1 & s2)
    if len(u) > 0:
        return len(i)/len(u)
    return 0
train['jaccard'] = train.apply(jaccard,axis=1)
test['jaccard'] = test.apply(jaccard,axis=1)

In [ ]:
np.array(train['is_duplicate'])

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
# Compute ROC curve and ROC area for each class
y = np.array(train['is_duplicate'])
scores = np.array(train['jaccard'])
fpr, tpr, thresholds = roc_curve(y, scores, pos_label=1)
roc_auc = roc_auc_score(y, scores)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
y.shape

In [ ]:
x.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
y = np.array(train['is_duplicate'])
x = np.array(train['jaccard']).reshape(-1,1)
my_model = LogisticRegression(random_state=0,fit_intercept=True).fit(x, y)
y_pred = my_model.predict_proba(x)[:,1]

In [ ]:
log_loss(y,y_pred)

In [ ]:
x_test = np.array(test['jaccard']).reshape(-1,1)
test['is_duplicate'] = my_model.predict_proba(x_test)[:,1]

In [ ]:
test.head(200)

In [ ]:
submission = test[['test_id','is_duplicate']]

In [ ]:
submission.head()
submission.to_csv('my_submission.csv',index=False)

In [ ]:
### Word2Vec - take sums and take cosine similarity
import gensim
train = pd.read_csv('train_cleaned.csv')
test = pd.read_csv('test_cleaned.csv')

In [ ]:

model = gensim.models.KeyedVectors.load_word2vec_format('./../models/GoogleNews-vectors-negative300.bin', binary=True)  



In [ ]:
def similarity(row):
    s1 = []
    s2 = []
    for w in row['question1_cleaned']:
        if model.__contains__(w):
            s1.append(w)
    for w in row['question2_cleaned']:
        if model.__contains__(w):
            s2.append(w)
    
    
    if len(s1) > 0 and len(s2) > 0:
        return model.n_similarity(s1,s2)
    return 0          
                       

In [ ]:
train.iloc[0]

In [ ]:
train['google_w2v_similarity'] = train.apply(similarity,axis=1)
test['google_w2v_similarity'] = test.apply(similarity,axis=1)

In [ ]:
train.describe()

In [ ]:
train['google_w2v_similarity'] =

In [ ]:
train.head()

In [ ]:
train.head()

In [ ]:
v = model['computer']

In [ ]:
v.shape

In [ ]:
v

In [ ]:
y = np.array(train['is_duplicate'])
scores = np.array(train['google_w2v_similarity'])
fpr, tpr, thresholds = roc_curve(y, scores, pos_label=1)
roc_auc = roc_auc_score(y, scores)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
y = np.array(train['is_duplicate'])
x = np.array(train[['google_w2v_similarity']])
my_model = LogisticRegression(random_state=0,fit_intercept=True).fit(x, y)
y_pred = my_model.predict_proba(x)[:,1]

In [ ]:
log_loss(y,y_pred)

In [ ]:
x_test = np.array(test['google_w2v_similarity']).reshape(-1,1)
test['is_duplicate'] = my_model.predict_proba(x_test)[:,1]

In [ ]:
train.head()

In [ ]:
np.correlate(train['google_w2v_similarity'],train['jaccard'])